# Se serializa el modelo

In [ ]:
import pickle

# Guardar el modelo con Keras
model_path = "best_model.keras"

# Serializar la ruta al modelo
with open("modelo_probabilidad.pkl", "wb") as f:
    pickle.dump(model_path, f)






# Se carga el modelo

In [3]:
import pickle
from tensorflow.keras.models import load_model
# Para cargar el modelo desde el archivo pickle
with open("modelo_probabilidad.pkl", "rb") as f:
    loaded_model_path = pickle.load(f)

model = load_model(loaded_model_path)

print(type(model))

<class 'keras.src.models.sequential.Sequential'>


c:\Users\samue\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 34 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


# Se utiliza el modelo

In [ ]:
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import load_model

# Cargar el modelo entrenado
best_model = load_model("best_model.keras")

# Cargar el scaler guardado
scaler = joblib.load('scaler.pkl')

# Lista de características utilizadas en el modelo
selected_features = [
    'emp_length', 'home_ownership', 'annual_inc', 'delinq_2yrs', 'open_acc',
    'total_acc', 'tot_cur_bal'
]

# Mapeo manual de categorías a valores numéricos
categoria_a_numero = {
    'RENT': 0,
    'OWN': 1,
    'MORTGAGE': 2,
    'OTHER': 3
}

# Función para procesar un vector de características y predecir la probabilidad de incumplimiento
def predecir_incumplimiento(vector_caracteristicas):
    """
    Procesa un vector de características y devuelve la probabilidad de incumplimiento.

    Parámetros:
    - vector_caracteristicas: Lista o array con los valores de las características en el siguiente orden:
        ['emp_length', 'home_ownership', 'annual_inc', 'delinq_2yrs', 'open_acc', 'total_acc', 'tot_cur_bal']

    Retorna:
    - probabilidad_incumplimiento: Probabilidad de que el cliente incumpla (valor entre 0 y 1).
    """
    # Convertir el vector a un DataFrame para facilitar el procesamiento
    df = pd.DataFrame([vector_caracteristicas], columns=selected_features)

    # Aplicar el mapeo manual a la columna 'home_ownership'
    df['home_ownership'] = df['home_ownership'].map(categoria_a_numero).fillna(-1)  # Usar -1 para categorías desconocidas

    # Convertir las columnas numéricas a tipo float
    df = df.astype(float)

    # Aplicar la normalización usando el scaler guardado
    vector_normalizado = scaler.transform(df)

    # Realizar la predicción con el modelo
    probabilidad_incumplimiento = best_model.predict(vector_normalizado)[0][0]

    return probabilidad_incumplimiento




vector_ejemplo = [
        0, "MORTGAGE", 50.0, 3.0, 12.0, 2.0, 10000.0
    ]  # emp_length, home_ownership, annual_inc, delinq_2yrs, open_acc, total_acc, tot_cur_bal

    # Predecir la probabilidad de incumplimiento
probabilidad = predecir_incumplimiento(vector_ejemplo)
print(f"🔹 Probabilidad de incumplimiento: {probabilidad:.4f}")